### Семинар 3. Языковые модели (N-gram)

В этом семинаре мы построим простейшую языковую модель генерации анекдотов. Датасет взят [отсюда](https://t.me/NeuralShit/2321).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm.auto import tqdm
%matplotlib inline

In [2]:
with open('anek.txt', 'r') as f:
    aneki = f.read().strip().replace('<|startoftext|>', '').split('\n\n')

In [3]:
len(aneki), type(aneki)

(124155, list)

In [4]:
aneki[42]

'Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.'

### Токениация

Реализуем два варианта токенизации: обычную по словам и BPE. В дальнейшем будем их сравнивать.

In [5]:
from bpe import Encoder

In [6]:
# pct_bpe - proportion of tokens that obtained via BPE. Other are the most frequent words.
encoder = Encoder(50000, ngram_max=6, pct_bpe=0.95)
encoder.fit(aneki)

In [7]:
len(encoder.bpe_vocab)

47500

In [8]:
list(encoder.bpe_vocab.items())[:10], list(encoder.bpe_vocab.items())[-10:]

([('__eow', 2500),
  ('__sow', 2501),
  ('о', 2502),
  ('а', 2503),
  ('е', 2504),
  ('и', 2505),
  ('т', 2506),
  ('н', 2507),
  ('р', 2508),
  ('с', 2509)],
 [('ньюто', 49990),
  ('ньютон', 49991),
  ('геи', 49992),
  ('дилы', 49993),
  ('одилы', 49994),
  ('абар', 49995),
  ('заха', 49996),
  ('еревни', 49997),
  ('енятьс', 49998),
  ('изюм', 49999)])

In [9]:
aneki[42]

'Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.'

In [10]:
print(next(encoder.transform(aneki[42])))

[38]


In [11]:
example = aneki[42]
print(encoder.tokenize(example))
print(next(encoder.transform([example])))
print(next(encoder.inverse_transform(encoder.transform([example]))))

['кто', 'сказал', ',', 'что', 'солдат', '__sow', 'мечтае', 'т', '__eow', 'стать', '__sow', 'генера', 'лом', '__eow', '?', 'солдат', '__sow', 'мечтае', 'т', '__eow', 'стать', '__sow', 'хлебо', 'резо', 'м', '__eow', '.']
[59, 172, 2, 9, 1509, 2501, 20745, 2506, 2500, 385, 2501, 15260, 3297, 2500, 23, 1509, 2501, 20745, 2506, 2500, 385, 2501, 25012, 26588, 2441, 2500, 3]
кто сказал , что солдат мечтает стать генералом ? солдат мечтает стать хлеборезом .


In [12]:
def tokenize_bpe(text):
    tokenized = encoder.tokenize(text)
    clear_tokenized = []
    first = False
    saw_eow = True
    for token in tokenized:
        if token == '__sow':
            saw_eow = False
            first = True
            continue
        elif token == '__eow':
            saw_eow = True
            continue
        else:
            if first or saw_eow:
                clear_tokenized.append(token)
                first = False
            else:
                clear_tokenized.append('##' + token)
    return clear_tokenized


def tokenize(text):
    reg = re.compile(r'\w+')
    return reg.findall(text.lower())

In [13]:
tokenize_bpe(example)

['кто',
 'сказал',
 ',',
 'что',
 'солдат',
 'мечтае',
 '##т',
 'стать',
 'генера',
 '##лом',
 '?',
 'солдат',
 'мечтае',
 '##т',
 'стать',
 'хлебо',
 '##резо',
 '##м',
 '.']

### N-граммная языковая модель

Языковая модель – это вероятностная модель, которая считает вероятность последовательности токенов $P(w_1, \dots, w_T)$. Так как оценивать совместную вероятность в лоб тяжело, обычно ее разбивают на произведение условных вероятностей. 

$$
P(w_1, \dots, w_T) = P(w_1)\prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_1)
$$ 

На практике такие условные вероятности сложно оценивать, когда текст очень длинный. Языковые модели лучше всего работают с небольшим контекстом. Для решения этой проблемы можно явно ограничить длину контекста, записав такое предположение
$$
P(w_i \mid w_{i-1}, \dots, w_1) \approx P(w_i \mid w_{i-1}, \dots, w_{i-n+1}).
$$

Данная модель называется __n-граммной языковой моделью__, так как оценивает вероятности только n-грамм токенов. Тогда итоговая вероятность последовательности токенов записывается вот так

$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n+1}).
$$

Для начало последовательности можно добавить специальные токены `[UNK]`, чтобы в условии всегда был контекст фиксированной длины.


В этом семинаре мы не будем ничего учить, наша модель будет счетной. Поэтому для начала нам надо посчитать, сколько раз встречается каждая n-грамма. В начало последовательности будем добавлять  `[UNK]`, а в конец – `[EOS]`. При генерации модель будет выдавать `[EOS]`, когда настанет время остановиться.

In [14]:
from collections import defaultdict, Counter

UNK, EOS = "[UNK]", "[EOS]"

def count_ngrams(lines, n, tokenize=tokenize):
    """
    Count how many times each word occured after (n - 1) previous words
    Input: a list of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    If the prefix is too short, it should be padded with [UNK].
    Add [EOS] at the end of each sequence and consider it as all other token
    """
    counts = defaultdict(Counter)

    for line in lines:
        tokenized = [UNK] * (n - 1) + tokenize(line) + [EOS]
        for i in range(n - 1, len(tokenized)):
            counts[tuple(tokenized[i-n+1:i])][tokenized[i]] += 1

    return counts

In [15]:
dummy_lines = aneki[-5:]
dummy_lines

['Последним раскрытым громким преступлением в Киеве было убийство Столыпина...',
 'Если бесконечное количество российских футболистов запустить на бесконечное количество футбольных полей и дать им бесконечное количество времени, то один из них когда-нибудь забьёт гол.',
 'На чемпионат мира по футболу от России нужно Юлию Самойлову отправлять, хоть какая-то надежда на победу будет.',
 'В целях профилактики от всего весной следует есть много чеснока. От женщин, кстати, тоже помогает.',
 'На моих глазах как-то две девушки затаскивали кавказца в машину. Они худенькие, а он здоровый такой, никак не хотел в машину лезть. Они попросили у меня помощи, сказали, что собаку надо в ветклинику отвезти.']

In [16]:
dummy_counts = count_ngrams(dummy_lines, n=3)

In [18]:
dummy_counts

defaultdict(collections.Counter,
            {('[UNK]',
              '[UNK]'): Counter({'последним': 1, 'если': 1, 'на': 2, 'в': 1}),
             ('[UNK]', 'последним'): Counter({'раскрытым': 1}),
             ('последним', 'раскрытым'): Counter({'громким': 1}),
             ('раскрытым', 'громким'): Counter({'преступлением': 1}),
             ('громким', 'преступлением'): Counter({'в': 1}),
             ('преступлением', 'в'): Counter({'киеве': 1}),
             ('в', 'киеве'): Counter({'было': 1}),
             ('киеве', 'было'): Counter({'убийство': 1}),
             ('было', 'убийство'): Counter({'столыпина': 1}),
             ('убийство', 'столыпина'): Counter({'[EOS]': 1}),
             ('[UNK]', 'если'): Counter({'бесконечное': 1}),
             ('если', 'бесконечное'): Counter({'количество': 1}),
             ('бесконечное',
              'количество'): Counter({'российских': 1,
                      'футбольных': 1,
                      'времени': 1}),
             ('количе

In [17]:
dummy_counts[('громким', 'преступлением')]

Counter({'в': 1})

In [384]:
dummy_counts[(UNK, UNK)]

Counter({'последним': 1, 'если': 1, 'на': 2, 'в': 1})

Теперь мы можем оценить вероятности, используя посчитанные n-граммы.

$$ P(w_i | prefix) = \frac{Count(prefix, w_i)}{\sum_{w \in V} Count(prefix, w)} $$

In [19]:
class NGramLanguageModel:
    def __init__(self, corpus, n=3, tokenize=tokenize):

        counts = count_ngrams(corpus, n, tokenize=tokenize)
        self.n = n

        self.probs = defaultdict(Counter)

        # calculate the probabilities using the formula above
        for prefix, token_count in counts.items():
            token_sum = sum(token_count.values())
            for token, count in token_count.items():
                self.probs[prefix][token] = count / token_sum
        
    def process_prefix(self, prefix):
        if self.n == 1:
            prefix = []
        else:
            prefix = prefix[-(self.n - 1):]
            prefix = [UNK] * (self.n - 1 - len(prefix)) + prefix
            
        return prefix

    def get_tokens_and_probs(self, prefix):
        prefix = self.process_prefix(prefix)

        possible_tokens = self.probs[tuple(prefix)]

        tokens = list(possible_tokens.keys())
        probs = list(possible_tokens.values())

        return tokens, probs
    
    def get_token_prob(self, token, prefix):
        prefix = self.process_prefix(prefix)

        prob = self.probs[tuple(prefix)].get(token, 0)
        return prob

Наконец, мы можем использовать полученную модель для генерации анекдотов.

In [20]:
lm = NGramLanguageModel(aneki, n=3, tokenize=tokenize_bpe)

Процесс генерации всегда авторегрессионный. Это значит, что выход модели на предыдущем шаге поступает на вход следующего. Таким образом можно бесконечно генерировать текст (ну или до тех пор, пока модель не выдаст [EOS]).

Для выбора одного токена из всех возможных вариантов существует огромное количество техник. Например, можно брать самый вероятный или семплировать токен в соответствии с вероятностями. Более подробно обсудим это на 4 семинаре. Мы остановимся на втором подходе, чтобы каждый раз получались разные тексты.

$$w_{next} \sim \frac{P(w_{next} | prefix)}{\sum_{w} P(w | prefix)}$$

In [22]:
def get_next_token(lm, prefix):
    tokens, probs = lm.get_tokens_and_probs(prefix)

    next_token = np.random.choice(tokens, p=probs)
    return next_token

In [28]:
prefix = tokenize('мужчина')

for i in range(100):
    prefix += [get_next_token(lm, prefix)]
    if prefix[-1] == EOS or len(lm.get_tokens_and_probs(prefix)[0]) == 0:
        break

print(' '.join(prefix))

мужчина меняет женщин , кстати , о которой я бы давно достиг ##ли в покое . [EOS]


### Оценка качества языковой модели: перплексия

Перплексия оценивает то, насколько хорошо модель предсказывает распределение данных. Она считатся по данной формуле:
$$
    {\mathbb{P}}(w_1 \dots w_T) = PPL(w_1, \dots, w_T)^{-\frac{1}{T}} = \left( \prod_i P(w_i \mid w_{i-1}, \dots, w_{i - n + 1})\right)^{-\frac{1}{T}},
$$

Можно заметить, что это в точности экспонента кросс-энтропии. Поэтому, чем меньше перплексия, тем лучше.

In [29]:
def perplexity(lm, lines, min_prob=10 ** -50., tokenize=tokenize):
    """
    :param min_prob: if P(w | ...) is smaller than min_prop, set it to min_prob.
    :returns: mean perplexity over the whole corpus
    """

    ppls = []
    for line in tqdm(lines):
        tokenized = tokenize(line)
        log_ppl = 0
        for i in range(len(tokenized)):
            log_ppl += np.log(max(
                min_prob,
                lm.get_token_prob(tokenized[i], tokenized[:i])
            ))
        ppls.append(np.exp(-log_ppl / len(tokenized)))

    return np.mean(ppls)

In [30]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Perplexities: ppx1=81.663 ppx3=1.142 ppx10=1.103


Теперь мы можем посчитать перплексию нашей модели.

In [35]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(aneki, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(train_lines, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  0%|          | 0/31039 [00:00<?, ?it/s]

N = 1, Perplexity = 12887014247997405467423213088777838380250562560.00000


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 2, Perplexity = 460881476685444201036843739570228114189619036160.00000


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 3, Perplexity = 3010794325323826519754827834236485472713017655296.00000


### Сглаживание вероятностей

Проблема заключается в следующем. Каждый раз, когда модель встречает в тестовом корпусе n-грамму, которой не было в тренировочном, она присваивает ей нулевую вероятность. Соответствено, все произведение зануляется, независимо от того, как выглядит остальной текст.

Один из способов обойти это – добавить сглаживание вероятностей ([сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing)). Сделаем вид, что мы видели каждую n-грамму хотя бы один раз. При наличии достаточно большого корпуса, это почти не поменяет распределения вероятностей, но зато позволит нам не взрывать перплексию.

$$ P(w_t \mid prefix) = \frac{Count(prefix, w_t) + \delta}{\sum_{w \in V} \big(Count(prefix, w) + \delta\big)} $$

In [36]:
class LaplaceLanguageModel(NGramLanguageModel):
    def __init__(self, corpus, n, delta=1.0, tokenize=tokenize):

        counts = count_ngrams(corpus, n, tokenize=tokenize)
        self.n = n
        self.vocab = set()
        for token_count in counts.values():
            self.vocab |= set(token_count.keys())

        self.probs = defaultdict(Counter)
        for prefix, token_count in counts.items():
            total = sum(token_count.values()) + delta * len(self.vocab)
            for token, count in token_count.items():
                self.probs[prefix][token] = (count + delta) / total

    def get_tokens_and_probs(self, prefix):
        # we want to spread some propability among all tokens
        
        tokens, probs = super().get_possible_next_tokens(prefix)
        
        left_prob = 1.0 - sum(probs)
        unseen_prob = left_prob / max(1, len(self.vocab) - len(tokens))
        
        unseen_tokens = self.vocab - set(tokens)

        return tokens + list(unseen_tokens), probs + [unseen_prob] * len(unseen_tokens)

    def get_token_prob(self, token, prefix):
        prob = super().get_token_prob(token, prefix)
        if prob > 0:
            return prob

        tokens, probs = super().get_tokens_and_probs(prefix)

        left_prob = max(1e-8, 1.0 - sum(probs))
        unseen_prob = left_prob / max(1, len(self.vocab) - len(tokens))

        return unseen_prob

In [37]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert sum(([dummy_lm.get_token_prob(w_i, ['l']) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [38]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  0%|          | 0/31039 [00:00<?, ?it/s]

N = 1, Perplexity = 39955.87579


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 2, Perplexity = 30058.30273


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 3, Perplexity = 70786.18871


In [39]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=1, tokenize=tokenize_bpe)
    ppx = perplexity(lm, test_lines, tokenize=tokenize_bpe)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

  0%|          | 0/31039 [00:00<?, ?it/s]

N = 1, Perplexity = 2695.94599


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 2, Perplexity = 3860.07277


  0%|          | 0/31039 [00:00<?, ?it/s]

N = 3, Perplexity = 15598.23201
